In [1]:
import sys
sys.path.append('/Users/jdli/Project/Korg.jl/Jorg/src/')

from jorg.statmech.species import Species, Formula

# Create Species objects
h_i = Species.from_string("H I")
print(h_i)

h_plus = Species.from_string("H+")
print(h_plus)

h2o = Species.from_string("H2O")
print(h2o)

# MOOG-style numeric codes are also supported
co = Species.from_string("0608")
print(co)

H I
H II
H2O
CO


In [2]:
c2_formula = Formula.from_string("C2")
print(c2_formula)

c2_species = Species(c2_formula, 2)
print(c2_species)

C2
C2(+2)


In [3]:
s = Species.from_string("H2O")
print(s)

print(s.get_atoms())
print(s.n_atoms)
print(s.mass)
print(s.is_molecule)

H2O
(1, 1, 8)
3
18.016
True


In [4]:
from jorg.statmech.partition_functions import create_default_partition_functions
from jorg.statmech.ionization import create_default_ionization_energies
from jorg.statmech.molecular import create_default_log_equilibrium_constants
from jorg.statmech.species import Species

partition_funcs = create_default_partition_functions()
ionization_energies = create_default_ionization_energies()
log_equilibrium_constants = create_default_log_equilibrium_constants()

fe_i = Species.from_string("Fe I")
print(partition_funcs[fe_i])

# Oxygen is atomic number 8
print(ionization_energies[8])

co = Species.from_string("CO")
print(log_equilibrium_constants[co])

ImportError: cannot import name 'create_default_partition_functions' from 'jorg.statmech.partition_functions' (/Users/jdli/Project/Korg.jl/Jorg/src/jorg/statmech/partition_functions.py)

In [7]:
import numpy as np
from jorg.statmech.chemical_equilibrium import chemical_equilibrium
from jorg.abundances import format_A_X
from jorg.statmech.species import Species
from jorg.statmech.partition_functions import create_default_partition_functions
from jorg.statmech.ionization import create_default_ionization_energies
from jorg.statmech.molecular import create_default_log_equilibrium_constants

# Define plasma parameters
T = 5700.0  # K
nt = 1e15   # cm⁻³
model_atm_ne = 1e12 # cm⁻³

# Get solar abundances
abundances = format_A_X()
absolute_abundances = {Z: 10**(A_X - 12.0) for Z, A_X in abundances.items()}
#normalize so that sum(n(X)/n_tot) = 1
norm_factor = sum(absolute_abundances.values())
absolute_abundances = {Z: val/norm_factor for Z, val in absolute_abundances.items()}


partition_funcs = create_default_partition_functions()
ionization_energies = create_default_ionization_energies()
log_equilibrium_constants = create_default_log_equilibrium_constants()

# Calculate equilibrium
ne, number_densities = chemical_equilibrium(
    T, nt, model_atm_ne, absolute_abundances,
    ionization_energies, partition_funcs, log_equilibrium_constants
)

print(f"ne = {ne:.3e}")

h_i = Species.from_string("H I")
h_ii = Species.from_string("H II")
h_minus = Species.from_string("H-")
co = Species.from_string("CO")

print(f"n(H I) = {number_densities.get(h_i, 0):.3e}")
print(f"n(H II) = {number_densities.get(h_ii, 0):.3e}")
print(f"n(H-) = {number_densities.get(h_minus, 0):.3e}")
print(f"n(CO) = {number_densities.get(co, 0):.3e}")

ImportError: cannot import name 'format_A_X' from 'jorg.abundances' (/Users/jdli/Project/Korg.jl/Jorg/src/jorg/abundances.py)